# Objective Functions

This Jupyter notebook shows how we can use CIFY to get pre-defined common benchmark objective functions or define different types of custom `ObjectiveFunction`s. `ObjectiveFunction`s can be static or dynamic and can have only a single objective or contain multiple objectives. They can also have constraints on their boundaries or on vectors in the search space that can be static or dynamic. An objective function can contain one or more of these characteristics and CIFY makes it easy to add or remove characteristics. First we must import cify. We will also set a seed for the global random number generator that will be used by objects that use stochastic operations. This step is not necessary if you do not care for repeatability, however it is recommended, especially if you wish to use CIFY for research work. Once the seed is set, you can forget about it.

In [1]:
# To install cify via pip:
# ! pip install cify
import cify as ci

ci.set_seed(0)

CIFY: internal seed successfully set to: '0'


Once we have cify imported, we can begin to define an objective function that we wish to optimize. Generally, the flow of optimizing an objective function using CIFY can be described by defining the following:

1. The `ObjectiveFunction` we wish to optimize.
1. A `Collection` or multiple `Collection`s of `Agent`s for the `ObjectiveFunction`.
1. An `Algorithm` to solve the `ObjectiveFunction` using the defined `Collection` or `Collection`s.

Once these high-level components are defined, we can simply call the `iterate()` method of the algorithm. This method takes various parameters, and it is recommended to consult the API documentation page (by searching for `iterate()` or finding the page manually in `core.base_class.Algorithm`) for more information. For this tutorial, we will only use `iterate(500)`, where `500` is the number of iterations to perform.

You may also use the `execute()` method of the algorithm, it is essentially just a wrapper method for `iterate()`.

## Provided Benchmark Objective Functions

CIFY ships with a useful utility function `get_objective_function()` that returns an `ObjectiveFunction` object. For a list of provided benchmark objective functions check out the Benchmark Objective Functions page. An example usage of this function is shown below.

In [5]:
ci.get_objective_function('schwefel', ci.Optimization.Min, n_dimensions=5)

## Custom Objective Functions

We can also define custom objective functions. We will explore six characteristics of objective functions in CIFY whilst building an `ObjectiveFunction` that will have all six characteristics.

### 1. Unconstrained

An unconstrained single objective optimization (SOO) objective function is the most basic of all objective functions. We only need to give the function, optimization type and random number generator. If you do not care about repeatability, no random number generator needs to be passed as a default option will be used. First, we define the function we want to optimize. We'll only be using a simple exponential function for this example.

In [7]:
def exp_function(vector):
    total = 0
    for i in range(len(vector)):
        total += vector[i] ** 2
    return 2 ** total

**NOTE: Our function only takes a vector as a parameter!**

It is important to follow this standard as all the inner workings of the framework expect the function of an `ObjectiveFunction` object to only take a vector as a parameter. Once we've defined our function, we can create the `ObjectiveFunction` object. We'll pass our function, set minimization as our optimization type, pass `5` as our number of dimensions and pass the random number generator defined earlier. We'll create a new variable, `obj_func`, from our new `ObjectiveFunction` so that we can change it later on.

In [9]:
obj_func = ci.ObjectiveFunction(function=exp_function,
                                  optimization=ci.Optimization.Min,
                                  n_dimensions=5
                                  )

### 2. Boundary Constrained

What if we wish to add boundary constraints? There are two approaches to achieve this. We could redefine our `ObjectiveFunction` to have boundary constraints:

In [10]:
ci.ObjectiveFunction(function=exp_function,
                     optimization=ci.Optimization.Min,
                     n_dimensions=5,
                     bounds=[-10, 10]
                     )

Or we could set the bounds after creation:

In [11]:
obj_func.bounds = [-10, 10]

**NOTE: You only need to pass a list with two elements representing the lower and upper boundary constraints if they are the same for all dimensions.** 

If you want different bounds for some dimensions, you must pass a list of lower and upper bounds equal to the number of dimensions of the objective function. The following is a correct example of this:

In [12]:
obj_func.bounds = [[-1, 1], [-2, 2], [-3, 3], [-4, 4], [-5, 5]]

### 3. Adding Vector Constraints

Vector constraints can also be added that constrain elements of position vectors. Due to the potential for users wanting complex vector constraints, vector constraints are passed as a list of functions, where each function takes a vector as it's only parameter. For example, say we wish to limit the value of the 4th dimension (remember our `ObjectiveFunction` is in 5 dimensions) if it is greater than double the sum of all the other elements. Although this is a fairly esoteric example, we can easily define a function for this.

In [13]:
def limit_fourth(vector):
    # fourth dimension of vector must be less than the sum of all the other dimensions.
    return vector[3] < (vector[0] + vector[1] + vector[2] + vector[4])

**NOTE: Functions for vector constraints only take a vector as a parameter.**

Once we have defined our vector constraint functions, we can redefine the `ObjectiveFunction` like we did when defining boundary constraints passing a list of the functions as a parameter. Or we can just set the `vector_constraints` field of `obj_func`.

In [14]:
obj_func.vector_constraints = [limit_fourth]

### 4. Dynamic Objective Function

Dynamic objective functions are categorized by their changing `function` field. There are a couple ways to define dynamic objective functions. To demonstrate how each approach works, we'll define an `Algorithm` to optimize the objective function. We'll use a simple inertia weight particle swarm optimization algorithm for this example. We'll use a single swarm of 50 particles and set the control parameters `w` (weight), `c1` (first acceleration coefficient) and `c2` (second acceleration coefficient) to safe values that should result in convergence on an optimal solution. For more on importing, defining and using algorithms, check out the algorithms tutorial notebook.

In [15]:
from cify.si.pso.algorithm import InertiaWeightPSO

pso = InertiaWeightPSO(obj_func=obj_func,
                       swarms=[ci.get_swarm(50, obj_func)],
                       velocity_params={'w': 0.72, 'c1':1.4, 'c2':1.4}
                       )

The first approach is to redefine the `function` field of your `ObjectiveFunction` after each iteration or number of iterations. This is more of a "manual" approach compared to what we'll look at shortly.

In [16]:
func_1 = lambda vector: abs(vector[0] + vector[1] + vector[2] + vector[3] + vector[4])
func_2 = lambda vector: vector[0] - vector[1] + vector[2] - vector[3] + vector[4]
func_3 = lambda vector: sum(vector) * vector.mean()

# Perform 10 iterations attempting to minimize the objective function defined previously.
pso.iterate(3)

# First change
obj_func.function = func_3
pso.iterate(3)

# Second change
obj_func.function = func_2
pso.iterate(3)

# Third change
obj_func.function = func_1
pso.iterate(3)

# Print statistics
pso.statistics

,best,worst,mean,stdev,global_optimum,n_evaluations
iteration,,,,,,
1,13.035604,2.211748e+13,4.467273e+11,3.095861e+12,13.035604,253
2,2.864392,2.486185e+07,5.655651e+05,3.488678e+06,2.864392,455
3,2.373651,7.628355e+11,1.526532e+10,1.067958e+11,2.373651,487
4,0.002189,1.027258e+01,1.703717e+00,2.478991e+00,0.000834,444
5,0.000324,1.467760e+01,2.379483e+00,3.093592e+00,0.000324,435
6,0.000402,1.877013e+01,2.075894e+00,3.800000e+00,0.000324,477
7,-7.694251,4.734108e+00,-9.952367e-02,2.683646e+00,-10.932584,487
8,-12.169194,3.030978e+00,-5.289125e+00,3.725139e+00,-12.169194,496
9,-22.476677,-2.244005e+00,-1.111163e+01,4.417766e+00,-22.476677,436


If your function references a value that you wish to change at each iteration, you can use the `dynamic_variables` field to define how attributes of the objective function must change at each iteration.

In [ ]:
bias = 1
incr_bias = lambda x: x + 1
changing_func = lambda vector: abs(vector[0] + incr_bias(bias) + vector[1] + vector[2] + vector[3] + vector[4])
# won't work ^, will increment at every particle evaluation

obj_func.function
# def dynamic_func(vector):
#     return vector + bias


### 5. Dynamic Vector Constraints

### 6. Multi- and Many- Objectives